# Scraping Untappd nach Regionen
Seit einigen Jahren gibt es Untappd, eine App in der Bierliebhaber ihre neuen Bier-Entdeckungen. Dieses Script erlaubt es, Informationen von Untappd zu scrapen. 

Zwei Dinge vorneweg: 
- 1. Ganz automatisiert läuft es nicht, der Login (Schritt 1) muss manuell erfolgen. 
- 2. Wer ohne Proxys arbeiten will braucht Geduld, oder wie es die Guinness-Werbung formuliert: «Good things come to those who can wait.»

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from datetime import datetime
from random import shuffle
import urllib3
urllib3.disable_warnings()

### Selenium Scraper

In [2]:
# Öffne Chrome Webdriver für Selenium
driver = webdriver.Chrome()
#wähle Land für Suche
country = 'switzerland'
# sende webdriver zur Website
driver.get('https://untappd.com/search?q='+country+'&type=brewery&sort=')


In [5]:
#logge dich manuell auf untappd ein!
#Mehr Suchergebnisse mit While-Loop anzeigen (maximal 1000)
while len(driver.find_element_by_xpath('//*[@id="slide"]/div/div[1]/div/div/div[3]/a').text) > 8:
    button = driver.find_element_by_xpath('//*[@id="slide"]/div/div[1]/div/div/div[3]/a') #identify search field
    driver.execute_script("arguments[0].scrollIntoView(true)", button) #scroll to the search
    button.click()
    time.sleep(random.randint(3, 10))

In [6]:
#Erstelle eine Liste für alle Brauereien
breweries = driver.find_elements_by_class_name('beer-item')

In [7]:
#Loope durch alle Brauereien, greife erste Infos ab.
#Diesen Datensatz brauchen wir später, um alle Biere pro Brauerei aufzuspüren
data = []
for brewery in breweries:
    dic = {
    'name':0,
    'country':0,
    'kind':0,
    'no_beers':0,
    'no_ratings':0,
    'rating':0,
    'link':0,
    'date_brewery':0
                }
    print(brewery.find_element_by_class_name('name').text)
    dic['name'] = brewery.find_element_by_class_name('name').text
    dic['country'] = brewery.find_elements_by_class_name('style')[0].text
    dic['kind'] = brewery.find_elements_by_class_name('style')[1].text
    dic['no_beers'] = brewery.find_element_by_class_name('abv').text
    dic['no_ratings'] = brewery.find_element_by_class_name('ibu').text
    dic['rating'] = brewery.find_element_by_class_name('num').text
    dic['link']= brewery.find_element_by_class_name('label').get_attribute('href')
    
    now = datetime. now()
    dt_string = now. strftime("%d/%m/%Y %H:%M:%S")
    dic['date_brewery'] = dt_string
    data.append(dic)
df_breweries = pd.DataFrame(data)
df_breweries.head()

Brasserie des Franches-Montagnes (BFM)
Feldschlösschen
Brauerei Locher
Trois Dames
WhiteFrontier
Doppelleu Brauwerkstatt
Brauerei Eichhof
La Nébuleuse
Valaisanne
Calanda
Brasserie Docteur Gab's
Schützengarten
Rugenbräu
Heineken (Switzerland)
Bier Factory BREWERY & TAPROOM
Emdbräu
Turbinenbräu
Falken
S&A Brewing
Rosengarten
Kitchen Brew
Hoppy People
Fischerstube
Bière du Boxer
Altes Tramdepot
Felsenau
Blackwell Brewery
Birrificio San Martino
Müller Bräu
Ticino Brewing Company
Unser Bier
À tue-tête
Brauerei Baar
523
BierVision Monstein
Erusbacher & Paul AG
Kloster Fischingen
Ramseier Suisse
Officina Della Birra
Brauerei Uster
Frères Papinot
Bieraria Tschlin
Brauerei Adler
Dr. Brauwolf
Sonnenbräu
Egger Bier Worb
LägereBräu
7 Peaks
Haarige Kuh Brauerei
Sudwerk Brauerei
Les Brasseurs
Cinq 4000
Coop
Rathaus Brauerei Luzern
Amboss
Barbière
Öufi Brauerei Solothurn
Brasserie du Virage
JungfrauBräu
Zermatt Matterhorn Brauerei
Brauerei Luzern
Simmentaler Braumanufaktur
Burgdorfer Gasthausbrauerei

Brasserie Urbaine Genevoise
Bière Buse
Brauerei Thun AG
Full House Brewery
Bannwald Brewery
Schori's Beer
Ulf & The Wolf
Brauerei BoGe
Bieruri
Bières Des Minibulles
L'Agneau à Trois Pattes
Sankt Johanns Bräu
Stögu-Bier
Stozi-Bräu
La Prévert
Stägi Bräu
Sonnrain Brauerei
Tucho Brew
Penny Farthing
Braubär Brauerei
3 Finger Joe
Herrenberg Bräu
Chrüpfe Bier
La Diablesse
Brasserie Du Plombier
Fäze Bräu
Zipfelbier
la Collégiale
Haxli Schwändi Bräu
Landhotel Hirschen
Brasserie de l'Ours
BrauWerk
PeRo Brauerei
Kägi Bräu
Riminy
Courtysane
Brummbacher
Hof-Braui
Wilde Maa
Brauerei Vitznau
Les Deux Souris
Guldibräu - 47grad Nord
Don Nino
Grain Zurich
Brasserie Artisanale Du Mât
la Faougueuse
8406
Muschtibräu
Heidnisch Bier
Joranbräu GmbH
BlackDot Cider
Rütihöfler Bräu
Magie Bräu
165
Festbierbrauerei Stollo's Best
Viking Brew Lab
La Brassée Du Chevrier
Äigebraij
Brasserie du Cerbère
Brasserie du Poyet
Köchli Steinmaur
Pension Gimmelwald
Bierliebe
Brews Brothers
Braugarage Reinach
Sohnemann
Löwen Gar

,name,country,kind,no_beers,no_ratings,rating,link,date_brewery
0,Brasserie des Franches-Montagnes (BFM),Switzerland,Micro Brewery,162 Beers,"138,634 Ratings",(3.97),https://untappd.com/brewery/4719,01/04/2020 10:18:05
1,Feldschlösschen,Switzerland,Macro Brewery,64 Beers,"83,339 Ratings",(3),https://untappd.com/brewery/3058,01/04/2020 10:18:06
2,Brauerei Locher,Switzerland,Regional Brewery,98 Beers,"62,204 Ratings",(3.26),https://untappd.com/brewery/217,01/04/2020 10:18:06
3,Trois Dames,Switzerland,Micro Brewery,85 Beers,"55,938 Ratings",(3.73),https://untappd.com/brewery/4720,01/04/2020 10:18:06
4,WhiteFrontier,Switzerland,Micro Brewery,120 Beers,"43,758 Ratings",(3.6),https://untappd.com/brewery/185140,01/04/2020 10:18:07


In [15]:
#Speichere / Lade die Daten 
#df_breweries.to_csv('schweizer_brauereien_untappd.csv',index=False)
df_breweries = pd.read_csv('schweizer_brauereien_untappd.csv')
df_breweries.head()

,name,country,kind,no_beers,no_ratings,rating,link,date_brewery
0,Brasserie des Franches-Montagnes (BFM),Switzerland,Micro Brewery,162 Beers,"138,634 Ratings",(3.97),https://untappd.com/brewery/4719,01/04/2020 10:18:05
1,Feldschlösschen,Switzerland,Macro Brewery,64 Beers,"83,339 Ratings",(3),https://untappd.com/brewery/3058,01/04/2020 10:18:06
2,Brauerei Locher,Switzerland,Regional Brewery,98 Beers,"62,204 Ratings",(3.26),https://untappd.com/brewery/217,01/04/2020 10:18:06
3,Trois Dames,Switzerland,Micro Brewery,85 Beers,"55,938 Ratings",(3.73),https://untappd.com/brewery/4720,01/04/2020 10:18:06
4,WhiteFrontier,Switzerland,Micro Brewery,120 Beers,"43,758 Ratings",(3.6),https://untappd.com/brewery/185140,01/04/2020 10:18:07


### Requests-Scraping, um Bier-Informationen zu erhalten

In [16]:
#Erstelle eine Liste mit allen Brauerei-Links
brewery_links = df_breweries.link.to_list()

In [19]:
#Loop über alle Brauerein
data = []
count = -1

for brewery_link in brewery_links:
    count+=1
    
    #Untappd mag keine Serien-Abfragen, deshalb sind Pausen notwendig
    if count in list(range(80,1000,80)):
        time.sleep(random.randint(300, 900))


    print(len(brewery_links)-count)
    dic = {
        'link':0,
        'location':0,
        'no_checkins_all_time':0,
        'no_users_all_time':0,
        'monthly':0,
        'beer_name':0,
        'beer_stye':0,
        'abv':0,
        'ibu':0,
        'beer_rating':0,
        'raters':0,
        'added':0,
        'beer_link':0
        }
    dic['link'] = brewery_link
    
    time.sleep(random.randint(20, 30))
    
    try:
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
        headers = {'User-Agent': user_agent}
        # Make web request for that URL and don't verify SSL/TLS certs
        response = requests.get(brewery_link+'/beer', headers=headers, verify=False)
        print(response)
        brewery = BeautifulSoup(response.text, 'html.parser')
        
        #print(brewery.find(class_='basic').find(class_='brewery').text.strip())
        dic['location'] = brewery.find(class_='basic').find(class_='brewery').text.strip()
        dic['no_checkins_all_time'] = brewery.find_all(class_='count')[0].text.strip()
        dic['no_users_all_time'] = brewery.find_all(class_='count')[1].text.strip()
        dic['monthly'] = brewery.find_all(class_='count')[2].text.strip()
    
        
        beers = brewery.find_all(class_='beer-item')
        
        for beer in beers:
            #print(beer.find(class_='name').text)
            dic['beer_name'] = beer.find(class_='name').text
            dic['beer_stye'] = beer.find(class_='style').text
            dic['abv'] = beer.find(class_='abv').text.strip()
            dic['ibu'] = beer.find(class_='ibu').text.strip()
            dic['beer_rating'] = beer.find(class_='num').text.strip()
            dic['raters']= beer.find(class_='raters').text.strip()
            dic['added']= beer.find(class_='date').text.strip()
            dic['beer_link'] =  'https://untappd.com' + beer.find(class_='label').attrs['href']
            #print(dic)
            data.append(dic.copy())
            #print(dic['beer_name'])
        
    except:
        data.append(dic.copy())
        print('error')
        continue
df_beers = pd.DataFrame(data)
df_beers.head()

1000
<Response [200]>
999
<Response [200]>
998
<Response [200]>
997
<Response [200]>
996
<Response [200]>
995
<Response [200]>
994
<Response [200]>
993
<Response [200]>
992
<Response [200]>
991
<Response [200]>
990
<Response [200]>
989
<Response [200]>
988
<Response [200]>
987
<Response [200]>
986
<Response [200]>
985
<Response [200]>
984
<Response [200]>
983
<Response [200]>
982
<Response [200]>
981
<Response [200]>
980
<Response [200]>
979
<Response [200]>
978
<Response [200]>
977
<Response [200]>
976
<Response [200]>
975
<Response [200]>
974
<Response [200]>
973
<Response [200]>
972
<Response [200]>
971
<Response [200]>
970
<Response [200]>
969
<Response [200]>
968
<Response [200]>
967
<Response [200]>
966
<Response [200]>
965
<Response [200]>
964
<Response [200]>
963
<Response [200]>
962
<Response [200]>
961
<Response [200]>
960
<Response [200]>
959
<Response [200]>
958
<Response [200]>
957
<Response [200]>
956
<Response [200]>
955
<Response [200]>
954
<Response [200]>
953
<Respons

<Response [200]>
609
<Response [200]>
608
<Response [200]>
607
<Response [200]>
606
<Response [200]>
605
<Response [200]>
604
<Response [200]>
603
<Response [200]>
602
<Response [200]>
601
<Response [200]>
600
<Response [200]>
599
<Response [200]>
598
<Response [200]>
597
<Response [200]>
596
<Response [200]>
595
<Response [200]>
594
<Response [200]>
593
<Response [200]>
592
<Response [200]>
591
<Response [200]>
590
<Response [200]>
589
<Response [200]>
588
<Response [200]>
587
<Response [200]>
586
<Response [200]>
585
<Response [200]>
584
<Response [200]>
583
<Response [200]>
582
<Response [200]>
581
<Response [200]>
580
<Response [200]>
579
<Response [200]>
578
<Response [200]>
577
<Response [200]>
576
<Response [200]>
575
<Response [200]>
574
<Response [200]>
573
<Response [200]>
572
<Response [200]>
571
<Response [200]>
570
<Response [200]>
569
<Response [200]>
568
<Response [200]>
567
<Response [200]>
566
<Response [200]>
565
<Response [200]>
564
<Response [200]>
563
<Response [20

<Response [200]>
218
<Response [200]>
217
<Response [200]>
216
<Response [200]>
215
<Response [200]>
214
<Response [200]>
213
<Response [200]>
212
<Response [200]>
211
<Response [200]>
210
<Response [200]>
209
<Response [200]>
208
<Response [200]>
207
<Response [200]>
206
<Response [200]>
205
<Response [200]>
204
<Response [200]>
203
<Response [200]>
202
<Response [200]>
201
<Response [200]>
200
<Response [200]>
199
<Response [200]>
198
<Response [200]>
197
<Response [200]>
196
<Response [200]>
195
<Response [200]>
194
<Response [200]>
193
<Response [200]>
192
<Response [200]>
191
<Response [200]>
190
<Response [200]>
189
<Response [200]>
188
<Response [200]>
187
<Response [200]>
186
<Response [200]>
185
<Response [200]>
184
<Response [200]>
183
<Response [200]>
182
<Response [200]>
181
<Response [200]>
180
<Response [200]>
179
<Response [200]>
178
<Response [200]>
177
<Response [200]>
176
<Response [200]>
175
<Response [200]>
174
<Response [200]>
173
<Response [200]>
172
<Response [20

,link,location,no_checkins_all_time,no_users_all_time,monthly,beer_name,beer_stye,abv,ibu,beer_rating,raters,added,beer_link
0,https://untappd.com/brewery/4719,"Saignelégier, Jura Switzerland","179,382","77,977",487,Abbaye de Saint Bon-Chien,Sour - Other,11% ABV,N/A IBU,(4.17),"27,817 Ratings",Added 10/07/12,https://untappd.com/b/brasserie-des-franches-m...
1,https://untappd.com/brewery/4719,"Saignelégier, Jura Switzerland","179,382","77,977",487,√225 Saison,Farmhouse Ale - Saison,6% ABV,18 IBU,(3.89),"21,837 Ratings",Added 11/28/12,https://untappd.com/b/brasserie-des-franches-m...
2,https://untappd.com/brewery/4719,"Saignelégier, Jura Switzerland","179,382","77,977",487,Cuvée Alex Le Rouge,Stout - Russian Imperial,10.276% ABV,N/A IBU,(3.82),"8,739 Ratings",Added 11/08/13,https://untappd.com/b/brasserie-des-franches-m...
3,https://untappd.com/brewery/4719,"Saignelégier, Jura Switzerland","179,382","77,977",487,La Salamandre,Bière de Garde,5.5% ABV,N/A IBU,(3.53),"4,148 Ratings",Added 11/04/11,https://untappd.com/b/brasserie-des-franches-m...
4,https://untappd.com/brewery/4719,"Saignelégier, Jura Switzerland","179,382","77,977",487,La Meule,Blonde Ale,6% ABV,2 IBU,(3.54),"3,825 Ratings",Added 12/18/10,https://untappd.com/b/brasserie-des-franches-m...


In [ ]:
#df_beers.to_csv('schweizer_biere_untappd.csv',index=False)

In [2]:
#df_breweries = pd.read_csv('schweizer_brauereien_untappd.csv')
#df_beers = pd.read_csv('schweizer_biere_untappd.csv')